In [1]:
import numpy as np
import pandas as pd

In [ ]:
inbound = pd.read_csv('data/inbound.csv')
outbound = pd.read_csv('data/outbound.csv')
inventory = pd.read_csv('data/inventory.csv')
operation = pd.read_csv('data/operationCost.csv')
material = pd.read_csv('data/materialMaster.csv')

In [101]:
inboundSG = inbound[inbound['PLANT_NAME'] == 'SINGAPORE-WAREHOUSE']
inboundSG.groupby(['INBOUND_DATE', 'MATERIAL_NAME']).agg(
    total_quantity=('NET_QUANTITY_MT', 'sum')
).reset_index().sort_values(by='INBOUND_DATE')

,INBOUND_DATE,MATERIAL_NAME,total_quantity
0,2023/11/07,MAT-0085,23.375
1,2023/11/07,MAT-0116,46.750
2,2023/11/07,MAT-0309,49.500
3,2023/11/13,MAT-0273,24.750
4,2023/11/13,MAT-0354,24.750
...,...,...,...
676,2024/12/02,MAT-0302,99.000
677,2024/12/11,MAT-0366,22.500
679,2024/12/20,MAT-0302,148.500
678,2024/12/20,MAT-0247,247.500


In [103]:
inboundChina = inbound[inbound['PLANT_NAME'] == 'CHINA-WAREHOUSE']
inboundChina.groupby(['INBOUND_DATE', 'MATERIAL_NAME']).agg(
    total_quantity=('NET_QUANTITY_MT', 'sum')
).reset_index().sort_values(by='INBOUND_DATE')

,INBOUND_DATE,MATERIAL_NAME,total_quantity
0,2023/11/03,MAT-0237,297.0
1,2023/11/09,MAT-0009,62.4
2,2023/11/09,MAT-0332,160.0
3,2023/11/11,MAT-0229,247.5
4,2023/11/13,MAT-0007,93.5
...,...,...,...
1435,2024/12/22,MAT-0339,25.5
1437,2024/12/24,MAT-0118,247.5
1438,2024/12/24,MAT-0232,99.0
1436,2024/12/24,MAT-0042,99.0


In [104]:
inventory.head()

,BALANCE_AS_OF_DATE,PLANT_NAME,MATERIAL_NAME,BATCH_NUMBER,UNRESRICTED_STOCK,STOCK_UNIT,STOCK_SELL_VALUE,CURRENCY
0,12/31/2023,CHINA-WAREHOUSE,MAT-0045,SCRAP,164,KG,211,CNY
1,12/31/2023,CHINA-WAREHOUSE,MAT-0193,6024,0,KG,87666,CNY
2,12/31/2023,CHINA-WAREHOUSE,MAT-0193,5755,70720,KG,414419,CNY
3,12/31/2023,CHINA-WAREHOUSE,MAT-0193,3142,12240,KG,71726,CNY
4,12/31/2023,CHINA-WAREHOUSE,MAT-0193,6734,2720,KG,15939,CNY


In [111]:
inventory_grouped = inventory.groupby(['MATERIAL_NAME']).agg(
    total_quantity=('UNRESRICTED_STOCK', 'sum'),
    total_cost=('STOCK_SELL_VALUE', 'sum')
).reset_index().sort_values(by='total_quantity', ascending=True)

inventory_grouped

,MATERIAL_NAME,total_quantity,total_cost
237,MAT-0238,0,84195
180,MAT-0181,0,84195
296,MAT-0297,0,331155
297,MAT-0298,0,55194
369,MAT-0370,0,368096
...,...,...,...
55,MAT-0056,28215650,86104095
98,MAT-0099,28287375,54299245
144,MAT-0145,35038575,376664694
150,MAT-0151,40258675,53581594


In [113]:
inventory_grouped.to_csv("data/inventory_grouped.csv", index=False)